## OSRM (Open Source Routing Machine)

**Author**: SADDIK Imad
<br/>
**Date**: 26/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Nearest service](#toc1_)    
  - [Setup](#toc1_1_)    
  - [Using the service](#toc1_2_)    
    - [General options](#toc1_2_1_)    
    - [Making the request](#toc1_2_2_)    
    - [Another example](#toc1_2_3_)    

---

# <a id='toc1_'></a>[Nearest service](#toc0_)

Use this service if you want to snap a coordinate to the street network. Read more about it [here](https://project-osrm.org/docs/v5.24.0/api/#nearest-service).

## <a id='toc1_1_'></a>[Setup](#toc0_)

I selected one point from the map and I will visualize it with the `Folium` package.

In [1]:
import folium

from folium import Map
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.Marker(location=[point.latitude, point.longitude],
                      popup='Point').add_to(folium_map)

    return folium_map


point_1 = Point(latitude=33.89264295626195, longitude=-5.500305816538693)
center_point = Point(latitude=33.89264295626195, longitude=-5.500305816538693)

folium_map = get_folium_map(center_point, [point_1], zoom_level=18)
folium_map

## <a id='toc1_2_'></a>[Using the service](#toc0_)

### <a id='toc1_2_1_'></a>[General options](#toc0_)

The general options are used in every OSRM service, read more about them [here](https://project-osrm.org/docs/v5.5.1/api/?language=cURL#general-options). Here is a table that summarizes the options with their description.


| Parameter | Description |
|-----------|-------------|
| service | One of the following values: `route`, `nearest`, `table`, `match`, `trip`, `tile` |
| version | Version of the protocol implemented by the service. `v1` for all OSRM 5.x installations |
| profile | Mode of transportation, is determined statically by the Lua profile that is used to prepare the data using `osrm-extract`. Typically `car`, `bike` or `foot` if using one of the supplied profiles. |
| coordinates | String of format `{longitude},{latitude};{longitude},{latitude};{longitude},{latitude} ...]` or `polyline({polyline})`. |
| format | Only `json` is supported at the moment. This parameter is optional and defaults to `json`. |

### <a id='toc1_2_2_'></a>[Making the request](#toc0_)

In [2]:
service = 'nearest'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

The nearest endpoint follows this format: `/nearest/v1/{profile}/{coordinates}?number={number}`.

When using this service, you should use only one point when providing the coordinates parameter.

In [3]:
import requests

coordinates = f'{point_1.longitude},{point_1.latitude}'
number_of_snap_points = 1

url = f"{host}/{service}/{version}/{profile}/{
  
    coordinates}?number={number_of_snap_points}"

response = requests.get(url)
response.status_code

SyntaxError: unterminated string literal (detected at line 6) (3662720242.py, line 6)

The `200` status code indicates that the request was successful. Next, let's retrieve the coordinates of the point snapped to the street network.

In [ ]:
data = response.json()
data.keys()

dict_keys(['code', 'waypoints'])

In [ ]:
waypoints = data['waypoints']
print(f"Number of waypoints: {len(waypoints)}")

Number of waypoints: 1


Let's extract the waypoints dynamically

In [ ]:
nearest_points = []
for waypoint in waypoints:
    latitude = waypoint['location'][1]
    longitude = waypoint['location'][0]
    nearest_points.append(Point(latitude=latitude, longitude=longitude))

nearest_points

[Point(latitude=33.892547, longitude=-5.500277)]

When adding the waypoint to the map, it is snapped to the nearest street, even if the original point was located on a building. Why is this necessary? To calculate the shortest path between two points, the algorithm requires the points to be on the street network rather than on buildings.

In [ ]:
folium_map = get_folium_map(center_point, [point_1], zoom_level=18)

for point in nearest_points:
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        color='blue',
                        fill=True,
                        fill_color='blue').add_to(folium_map)

folium_map

NameError: name 'nearest_points' is not defined

: 

### <a id='toc1_2_3_'></a>[Another example](#toc0_)

In this example, we will load a CSV file containing several points that are not located on the street network. We will iterate through these points and use the nearest service to find the closest points snapped to the street network for each one.

In [ ]:
import pandas as pd

data = pd.read_csv('../data/nearest_service_data.csv')
data.head()

,latitude,longitude
0,33.892618,-5.500416
1,33.892538,-5.500902
2,33.892312,-5.501063
3,33.891908,-5.501253
4,33.891870,-5.501506


In [ ]:
points = []
for _, row in data.iterrows():
    point = Point(latitude=row['latitude'], longitude=row['longitude'])
    points.append(point)

points[:5]

[Point(latitude=33.89261795639958, longitude=-5.500416069831112),
 Point(latitude=33.89253843494372, longitude=-5.500902451816623),
 Point(latitude=33.8923118737899, longitude=-5.501062653942593),
 Point(latitude=33.891908002407774, longitude=-5.501252523134825),
 Point(latitude=33.89186983153817, longitude=-5.501506176515667)]

Here are the locations of all the points displayed on the map.

In [ ]:
folium_map = get_folium_map(points[5], points, zoom_level=18)
folium_map

In [ ]:
def get_waypoints(point: Point) -> list[Point]:
    coordinates = f'{point.longitude},{point.latitude}'
    number_of_snap_points = 1

    url = f"{
        host}/{service}/{version}/{profile}/{coordinates}?number={number_of_snap_points}"

    response = requests.get(url)

    data = response.json()
    waypoints = [Point(latitude=point['location'][1], longitude=point['location'][0])
                 for point in data['waypoints']]

    return waypoints


waypoints = []
for point in points:
    waypoints.append(get_waypoints(point))

waypoints[:5]

[[Point(latitude=33.892523, longitude=-5.500383)],
 [Point(latitude=33.892517, longitude=-5.500844)],
 [Point(latitude=33.892356, longitude=-5.501059)],
 [Point(latitude=33.891886, longitude=-5.501148)],
 [Point(latitude=33.891806, longitude=-5.501512)]]

Using the nearest service, we observe that the black circle markers have been snapped to the street network.

In [ ]:
folium_map = get_folium_map(points[5], points, zoom_level=18)

for waypoint in waypoints:
    for point in waypoint:
        folium.CircleMarker(
            location=[point.latitude, point.longitude],
            radius=5,
            color='black',
            fill=True,
            fill_color='black',
        ).add_to(folium_map)

folium_map